In [20]:
import requests
import json
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import datetime
import os

In [29]:
# VARIABLES

# DCurl stores DukasCopy URL
DCurl = "https://www.dukascopy.com/swiss/english/home/?utm_source=freeserv"

# Stores the filepath to the pickle file where dukas data will be saved
DukasFp = "DukasCopyData.pkl"

# Stores the filepath to the pickle file where MetalsAPI data will be saved
MetalsFp = "MetalsData.pkl"

In [8]:
# Converts epoch time to GMT Time

def convertFromEpoch(t):
    return time.strftime('%Y-%d-%m %H:%M:%S', time.gmtime(t))

print(convertFromEpoch(1578448320))

2020-08-01 01:52:00


In [9]:
# Fetches the time from DukasCopy Webpage and converts into readable format

def getDukasTime(soup):
    tdata = soup.find_all("span", {"id": "timeUpdate"})
    dtxt = tdata[0].text
    
    date_time_obj = datetime.datetime.strptime(dtxt, ' %a, %d %b %Y %H:%M:%S GMT')
    
    return date_time_obj

In [11]:
# Takes in a new Dataframe and the filepath of the pickle file. Appends the data from new dataframe 
# into the pickle file. New pickle file created if it does not exist.

def updateData(NewFrame, filepath):
    if not os.path.isfile(filepath):
        pd.to_pickle(NewFrame, filepath)
        
    else:   
        df = pd.read_pickle(filepath)
        df = df.append(NewFrame)
        df.to_pickle(filepath)

In [4]:
# Kept outside for now so I don't abuse the API limit. 
# ADD BACK INTO BELOW FUNCTION BEFORE RUNNING OR IT WON'T WORK!!!!!!!!

MetalsApiResponse = requests.get("http://metals-api.com/api/latest?access_key=anbne4hpic0ev3c2ymdyscbk5emkoni00qurxhpyrcvbuh2bu1j1yugbqbl17avu&base=USD&symbols = XAU") 

In [10]:
# Function to fetch Gold Data from MetalsAPI. Returns a new DataFrame with most recent data

def getMetalsApiData(MetalsApiResponse):
    print("MetalsAPI Connection " + str(MetalsApiResponse.status_code))
    
    # Get the result json
    result = MetalsApiResponse.json()
    
    # Store results in dataframe
    df = pd.DataFrame(result)
    df["timestamp"] = df["timestamp"].map(convertFromEpoch)
    
    # Store only gold data and make time stamp the index
    df = df.loc[df.index == "XAU"]
    df = df.set_index("timestamp")
    
    return df
    
getMetalsApiData(MetalsApiResponse)

NameError: name 'MetalsApiResponse' is not defined

In [18]:
# Function to fetch Gold Data from DukasCopy. Returns a new DataFrame with most recent data.

def getDukasCopyData():
    browser = webdriver.Chrome()
    browser.get(DCurl)
    DChtml = browser.page_source
    soup = BeautifulSoup(DChtml, "lxml")
    
    # Finding the exact table with all the required data
    data = soup.find_all("table", {"id": "list"})
    
    # Creating and Modifying dataframe with the table 
    DCDataFrame = pd.read_html(str(data))[0]
    
    # Add a timestamp to the data
    DCDataFrame["timestamp"] = getDukasTime(soup)
    
    # Make the timestamp the index
    DCDataFrame = DCDataFrame.set_index("timestamp")
    
    # Filter it only to Gold Data
    DCDataFrame = DCDataFrame[DCDataFrame["Live"] == "XAU/USD"]
    
    browser.quit()
    return DCDataFrame

In [14]:
# Fetches Dukas data and saves it to the pickle file.

def fetchAndSaveDukas(dukasFilepath):
    df = getDukasCopyData()
    updateData(df, dukasFilepath)
    
# Fetches MetalsAPI data and saves it to the pickle file.
def fetchAndSaveMetals(metalsFilepath):
    df = getMetalsApiData(dummy) # dont forget to remove the dummy thing
    updateData(df, metalsFilepath)